In [108]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import re

In [164]:
traint = pd.read_csv('finaltrain.csv') 

test = pd.read_csv('finaltest.csv')

In [165]:
def missingcheck(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
    return missing_data


In [111]:
missingcheck(traint)

,Total,%
price,0,0.0
gift_type_gift_cluster,0,0.0
lsg_2_lsg_3,0,0.0
lsg_2,0,0.0
lsg_1_lsg_3,0,0.0
...,...,...
instock_Day,0,0.0
buy1buytime,0,0.0
volumes,0,0.0
uk_date2Month,0,0.0


In [167]:
test['volumes']=test['volumes'].replace(0,10)

In [168]:
train['volumes']=train['volumes'].replace(0,10)

In [169]:
test['gift_type']

0         842
1         407
2         631
3         992
4          43
         ... 
13514      31
13515      31
13516      31
13517     280
13518    1293
Name: gift_type, Length: 13519, dtype: int64

In [170]:
orignal_test = pd.read_csv('dataset/test.csv')

In [171]:
test_ID = orignal_test['gift_id']


# model time

In [172]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [173]:
y_train = traint.price

train= traint.drop(['price'],axis=1)

In [174]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmae= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_absolute_error", cv = kf))
    return(rmae)

In [175]:
GBoost = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [176]:
# model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=720,
#                               max_bin = 55, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb= lgb.LGBMRegressor(objective='regression',n_estimators=150, learning_rate=0.09, num_leaves=42, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

In [177]:
model_rf = RandomForestRegressor(n_estimators=100,max_features ='auto',)
from sklearn.model_selection import GridSearchCV


In [178]:
# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

In [179]:
# CV_rfc = GridSearchCV(estimator=model_rf, param_grid=param_grid, cv= 5)
# CV_rfc.fit(train,y_train)

In [180]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [181]:
# CV_rfc = GridSearchCV(estimator=model_rf, param_grid=random_grid, cv= 3)
# CV_rfc.fit(train,y_train)

In [182]:
# score = rmsle_cv(GBoost)
# print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [183]:
# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [184]:
# score = rmsle_cv(model_rf)
# print("RF score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [262]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [263]:
#averaged_models = AveragingModels(models = ( GBoost,model_rf))

averaged_models = AveragingModels(models = ( GBoost,model_rf,model_lgb))
#### withlog

In [187]:
# averaged_models.fit(train.values,np.log1p(y_train))

In [188]:
# avg_pred = np.expm1(averaged_models.predict(test.values))


In [189]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = avg_pred
# sub.to_csv('submissionlog2.csv',index=False)

In [190]:
# model_rf.fit(train,np.log1p(y_train))
# pred = np.expm1(model_rf.predict(test))

In [191]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = pred
# sub.to_csv('submissionRF.csv',index=False)

# log chal gya 

In [192]:
#y_train = np.log1p(y_train)

In [209]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [210]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [259]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [260]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,GBoost),
                                                 meta_model = lasso)

# score = rmsle_cv(stacked_averaged_models)
# print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [261]:
stacked_averaged_models.fit(train, np.log1p(y_train))
stacked_averaged_models_train_pred = np.expm1(stacked_averaged_models.predict(train))

stacked_averaged_models_pred = np.expm1(stacked_averaged_models.predict(test))
print(rmsle(y_train, stacked_averaged_models_train_pred))

ValueError: Index data must be 1-dimensional

In [197]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor

In [286]:
# estimators = [
#      ('rf', model_rf)]

In [287]:
# reg = StackingRegressor(
#      estimators=estimators,
#      final_estimator= model_rf)

In [289]:
# reg.fit(train.values, np.log1p(y_train))

In [284]:
# reg_pred = np.expm1(reg.predict(test.values))


In [290]:
# sub = pd.DataFrame()
# sub['gift_id'] = test_ID
# sub['price'] = reg_pred
# sub.to_csv('submissionstacking1.csv',index=False)

In [291]:
def rmsle(y, y_pred):
    return np.sqrt(mean_absolute_error(y, y_pred))

In [292]:
# stacked_averaged_models.fit(train.values, y_train)
# stacked_train_pred = stacked_averaged_models.predict(train.values)
# stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
# print(rmsle(y_train, stacked_train_pred))

In [206]:
model_rf.fit(train, np.log1p(y_train))
model_rf_train_pred = np.expm1(model_rf.predict(train))

model_rf_pred = np.expm1(model_rf.predict(test))
print(rmsle(y_train, model_rf_train_pred))

3.874510821806858


3.885730865544163

In [150]:
# model_lgb.fit(train, np.log1p(y_train))
# model_lgb_train_pred = np.expm1(model_lgb.predict(train))

# model_lgb_pred = np.expm1(model_lgb.predict(test))
# print(rmsle(y_train, model_lgb_train_pred))

6.010385321640361

In [293]:
# GBoost.fit(train, np.log1p(y_train))
# GBoost_train_pred = np.expm1(GBoost.predict(train))

# GBoost_pred = np.expm1(GBoost.predict(test))
# print(rmsle(y_train, model_lgb_train_pred))

6.010385321640361

In [296]:
model_avg = AveragingModels(models = ( GBoost,model_rf))

In [297]:
model_avg.fit(train, np.log1p(y_train))
model_avg_train_pred = np.expm1(model_avg.predict(train))

model_avg_pred = np.expm1(model_avg.predict(test))
print(rmsle(y_train, model_avg_train_pred))


5.078302370658987


In [298]:
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                              random_state =7, nthread = -1)


In [299]:
# model_avg2 = AveragingModels(models = ( GBoost,model_rf,model_xgb))

In [300]:
# model_avg2.fit(train, np.log1p(y_train))
# model_avg2_train_pred = np.expm1(model_avg2.predict(train))

# model_avg2_pred = np.expm1(model_avg2.predict(test))
# print(rmsle(y_train, model_avg2_train_pred))


5.077430905907544

In [301]:
# Hit and trial mf

In [306]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train, model_rf_train_pred*0.50 + model_avg_train_pred*0.50 ))

RMSLE score on train data:
4.4692260239285035


array([145.8254597 ,  78.05588109, 103.84067734, ...,  41.13535816,
        60.88385685,  54.6539869 ])

In [160]:
#model_avg_pred*0.10 + model_rf_pred*0.60 +

In [327]:
ensemble =  model_avg_pred*0.90 + model_rf_pred*0.10 + model_lgb_pred*0.05

In [328]:
sub = pd.DataFrame()
sub['gift_id'] = test_ID
sub['price'] = ensemble
sub.to_csv('submissionEnsemble.csv',index=False)